In [1]:
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML 
from collections import defaultdict
import pandas as pd
import numpy as np
import spacy
import requests
import re

In [25]:
url = 'https://www.sefaria.org/texts/Mishnah'
response = requests.get(url)
#Already checked that response gives us a 200 status_code
page = response.text
soup = BeautifulSoup(page, "lxml")
soup = soup.find_all('td')
tractates = []
chapters = []
mishnaot = []
texts = []
for i in range(0,63):
    tractate = soup[i].find(class_='en').text
    print(tractate)
    link = soup[i].find('a')
    link = 'https://www.sefaria.org'+link['href']
    link = link[:-1]
    chapter = 1
    while requests.get(link+str(chapter)+'?lang=en'):
        response = requests.get(link+str(chapter)+'?lang=en')
        page = response.text
        tractate_soup = BeautifulSoup(page, "lxml").find_all('p',class_='en')
        for mishnah,text in enumerate(tractate_soup):
            tractates.append(tractate)
            chapters.append(chapter)
            mishnaot.append(mishnah+1)
            texts.append(text.text)
        chapter+=1


 Berakhot
 Peah
 Demai
 Kilayim
 Sheviit
 Terumot
 Maasrot
 Maaser Sheni
 Challah
 Orlah
 Bikkurim
 Shabbat
 Eruvin
 Pesachim
 Shekalim
 Yoma
 Sukkah
 Beitzah
 Rosh Hashanah
 Taanit
 Megillah
 Moed Katan
 Chagigah
 Yevamot
 Ketubot
 Nedarim
 Nazir
 Sotah
 Gittin
 Kiddushin
 Bava Kamma
 Bava Metzia
 Bava Batra
 Sanhedrin
 Makkot
 Shevuot
 Eduyot
 Avodah Zarah
Pirkei Avot
 Horayot
 Zevachim
 Menachot
 Chullin
 Bekhorot
 Arakhin
 Temurah
 Keritot
 Meilah
 Tamid
 Middot
 Kinnim
 Kelim
 Oholot
 Negaim
 Parah
 Tahorot
 Mikvaot
 Niddah
 Makhshirin
 Zavim
 Tevul Yom
 Yadayim
 Oktzin


In [5]:
mishnah_df = pd.DataFrame(np.array([tractates,chapters,mishnaot,texts]).T)
mishnah_df.columns = ['tractate','chapter','mishnah','text']
mishnah_df.tractate = mishnah_df.tractate.apply(lambda row: row.strip())

In [3]:
def assign_seder(row):
    if row.tractate in ['Berakhot', 'Peah', 'Demai', 'Kilayim', 'Sheviit', 'Terumot',
                        'Maasrot', 'Maaser Sheni', 'Challah', 'Orlah', 'Bikkurim']:
        return 'Zeraim'
    elif row.tractate in ['Shabbat', 'Eruvin', 'Pesachim', 'Shekalim', 'Yoma', 'Sukkah',
                          'Beitzah', 'Rosh Hashanah', 'Taanit', 'Megillah', 'Moed Katan','Chagigah']:
        return 'Moed'
    elif row.tractate in ['Yevamot', 'Ketubot', 'Nedarim', 'Nazir', 'Sotah',
                          'Gittin', 'Kiddushin']:
        return 'Nashim'
    elif row.tractate in ['Bava Kamma', 'Bava Metzia', 'Bava Batra',
                          'Sanhedrin', 'Makkot', 'Shevuot', 'Eduyot', 'Avodah Zarah',
                          'Pirkei Avot', 'Horayot']:
        return 'Nezikin'
    elif row.tractate in ['Zevachim', 'Menachot', 'Chullin',
                          'Bekhorot', 'Arakhin', 'Temurah', 'Keritot', 'Meilah', 'Tamid',
                          'Middot', 'Kinnim']:
        return 'Kadoshim'
    elif row.tractate in ['Kelim', 'Oholot', 'Negaim', 'Parah',
                          'Tahorot', 'Mikvaot', 'Niddah', 'Makhshirin', 'Zavim', 'Tevul Yom',
                          'Yadayim', 'Oktzin']:
        return 'Toharot'
    
mishnah_df['seder'] = mishnah_df.apply(lambda row: assign_seder(row), axis = 1)

def clean_quote(row):
    text = row
    while re.match('(Äô)|(Äú)',row):
        text = re.sub('(,Äô)',"\''",row)
        text = re.sub('(,Äú)','"',row)      
    return text

mishnah_df['text'] = mishnah_df.text.apply(lambda row: clean_quote(row))



ValueError: Wrong number of items passed 4, placement implies 1

In [19]:
mishnah_df.text[30]

'One who is praying and makes a mistake, it is a bad sign for him. And if he is the messenger of the congregation (the prayer leader) it is a bad sign for those who have sent him, because one’s messenger is equivalent to one’s self. They said about Rabbi Hanina ben Dosa that he used to pray for the sick and say, “This one will die, this one will live.” They said to him: “How do you know?” He replied: “If my prayer comes out fluently, I know that he is accepted, but if not, then I know that he is rejected.” '

In [20]:
mishnah_df.to_csv('mishnah.csv')